<a href="https://colab.research.google.com/github/Cattum/INFOSCI-301-Team-Research/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# Data Processing for dataset 1
import pandas as pd

df = pd.read_csv('data_043155.csv')


df['Value'] = df['Value'].str.rstrip('%').astype(float)
df['Confidence Interval Low'] = df['Confidence Interval Low'].str.rstrip('%').astype(float)
df['Confidence Interval High'] = df['Confidence Interval High'].str.rstrip('%').astype(float)

state_summary = df.groupby(['State', 'Year']).agg({
    'Value': 'mean',
    'Confidence Interval Low': 'mean',
    'Confidence Interval High': 'mean'
}).reset_index()

state_summary.head()

,State,Year,Value,Confidence Interval Low,Confidence Interval High
0,Alabama,2018,17.662687,16.150746,19.186567
1,Alabama,2019,18.952239,17.516418,20.347761
2,Alabama,2020,17.532836,16.388060,18.671642
3,Alabama,2021,19.525373,16.914925,22.298507
4,Alaska,2018,14.603448,13.458621,15.768966


In [65]:
# Data Processing for dataset 2
df_1 = pd.read_csv("Unemployment in America Per US State.csv")

df_filtered = df_1[df_1["Year"].between(2018, 2021)]

df_filtered["Percent (%) of Labor Force Unemployed in State/Area"] = (
    df_filtered["Percent (%) of Labor Force Unemployed in State/Area"]
    .replace({",": ""}, regex=True)
    .astype(float)
)

df_unemployment_rate = (
    df_filtered
    .groupby(["State/Area", "Year"])["Percent (%) of Labor Force Unemployed in State/Area"]
    .mean()
    .reset_index()
)

df_unemployment_rate = df_unemployment_rate.sort_values(by=["State/Area", "Year"]).reset_index(drop=True)

df_unemployment_rate.head()


<ipython-input-65-57ed148afd91>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,State/Area,Year,Percent (%) of Labor Force Unemployed in State/Area
0,Alabama,2018,3.941667
1,Alabama,2019,3.183333
2,Alabama,2020,6.425000
3,Alabama,2021,3.366667
4,Alaska,2018,6.041667


In [47]:
print(len(state_summary))

202


In [66]:
# Dataset Intersection



unemployment_data = df_unemployment_rate[['State/Area', 'Year', 'Percent (%) of Labor Force Unemployed in State/Area']]

unemployment_data.head()

unemployment_data.rename(columns={
    "State/Area": "State",
    "Percent (%) of Labor Force Unemployed in State/Area": "Unemployment Rate"
}, inplace=True)

combined_data = pd.merge(state_summary, unemployment_data, on=["State", "Year"], how="inner")

combined_data.head(10)

,State,Year,Value,Confidence Interval Low,Confidence Interval High,Unemployment Rate
0,Alabama,2018,17.662687,16.150746,19.186567,3.941667
1,Alabama,2019,18.952239,17.516418,20.347761,3.183333
2,Alabama,2020,17.532836,16.388060,18.671642,6.425000
3,Alabama,2021,19.525373,16.914925,22.298507,3.366667
4,Alaska,2018,14.603448,13.458621,15.768966,6.041667
5,Alaska,2019,15.175862,14.086207,16.255172,5.591667
6,Alaska,2020,13.420000,12.586667,14.240000,8.308333
7,Alaska,2021,16.436667,14.283333,18.723333,6.433333
8,Arizona,2018,14.986667,13.786667,16.240000,4.808333
9,Arizona,2019,15.906667,14.773333,17.053333,4.808333


In [67]:
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

combined_data['State_Code'] = combined_data['State'].map(us_state_abbrev)


In [69]:
import plotly.graph_objects as go

# Preprocessing Data
zmin = combined_data['Value'].min()
zmax = combined_data['Value'].max()
size_max = combined_data['Unemployment Rate'].max()
years = sorted(combined_data['Year'].unique())

fig = go.Figure()

traces = []

for year in years:
    df_year = combined_data[combined_data['Year'] == year]

    # Mental health
    fig.add_trace(
        go.Choropleth(
            locations=df_year['State_Code'],
            z=df_year['Value'],
            locationmode="USA-states",
            colorscale="tealrose",
            zmin=zmin,
            zmax=zmax,
            colorbar_title="Mental Health Problems Rate",
            text=df_year['State'],
            showscale=True,
            name=f"{year} - Mental Health",
            visible=(year == years[0])
        )
    )


    # Unemployment
    fig.add_trace(
        go.Scattergeo(
            locations=df_year['State_Code'],
            locationmode="USA-states",
            text=df_year['Unemployment Rate'].apply(lambda x: f"Unemployment Rate: {x:.2f}%"),
            marker=dict(
                opacity=0.5,
                size=df_year['Unemployment Rate'] * 1.3,
                sizemode='area',
                sizeref=2. * size_max / (40**2),
                sizemin=4,
                color=df_year['Unemployment Rate'],
                colorscale="tealrose",
                showscale=False,
                line=dict(width=3, color='white')
            ),
            name=f"{year} - Unemployment",
            visible=(year == years[0])

        )
    )

# Time Slider
steps = []
for i, year in enumerate(years):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Mental Health Problems and Unemployment Rate in US States (2018-2021)"}],
        label=str(year)
    )
    step["args"][0]["visible"][2*i] = True
    step["args"][0]["visible"][2*i + 1] = True
    steps.append(step)


# Layout

fig.update_layout(
    title="Mental Health Problems and Unemployment Rate in US States (2018-2021)",
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        showcoastlines=True,
        coastlinecolor="Black",
    ),
    sliders=[dict(
        active=0,
        steps=steps,
        x=0.1,
        y=0,
        xanchor="left",
        yanchor="top",
        len=0.8
    )],

)

fig.show()


In [70]:
# Color switchable version

import plotly.graph_objects as go
import pandas as pd
import numpy as np


colorscales = {
    "tealrose": "tealrose",
    "sunsetdark": "sunsetdark",
    "delta": "delta"
}


fig = go.Figure()
trace_map = {}
trace_index = 0

for c_idx, (cs_name, cs_value) in enumerate(colorscales.items()):
    trace_map[cs_name] = []
    for y_idx, year in enumerate(years):
        df_year = combined_data[combined_data["Year"] == year]


        fig.add_trace(go.Choropleth(
            locations=df_year['State_Code'],
            z=df_year['Value'],
            locationmode="USA-states",
            colorscale=cs_value,
            zmin=zmin,
            zmax=zmax,
            colorbar_title="Mental Health Problems Rate",
            text=df_year['State'],
            showscale=(y_idx == 0),

        ))
        trace_map[cs_name].append(trace_index)
        trace_index += 1


        fig.add_trace(go.Scattergeo(
            locations=df_year['State_Code'],
            locationmode="USA-states",
            text=df_year['Unemployment Rate'],
            marker=dict(
                opacity=0.5,
                size=df_year['Unemployment Rate'] * 1.3,
                sizemode='area',
                sizeref=2. * size_max / (40**2),
                sizemin=4,
                color=df_year['Unemployment Rate'],
                colorscale=cs_value,
                showscale=False,
                line=dict(width=2, color='white')
            ),
            name=f"{cs_name}-{year}-Bubble",
            visible=(cs_name == "delta" and year == 2018)
        ))
        trace_map[cs_name].append(trace_index)
        trace_index += 1

steps = []
for i, year in enumerate(years):
    step = {
        "method": "update",
        "args": [{"visible": [False] * len(fig.data)}],
        "label": str(year)
    }
    for cs_name, indices in trace_map.items():
        if cs_name == "delta":
            step["args"][0]["visible"][2*i] = True
            step["args"][0]["visible"][2*i+1] = True
    steps.append(step)


buttons = []
for cs_name in colorscales.keys():
    visibility = [False] * len(fig.data)
    idx_list = trace_map[cs_name]
    visibility[idx_list[0]] = True
    visibility[idx_list[1]] = True
    for j in range(len(years)):
        fig.data[idx_list[2*j]].showscale = (j == 0)
    button = dict(
        label=cs_name,
        method="update",
        args=[{"visible": visibility}]
    )
    buttons.append(button)


fig.update_layout(
    title="Mental Health Problems and Unemployment Rate in US States (2018-2021)",
    geo=dict(
        scope="usa",
        projection_type="albers usa",
        showcoastlines=True,
        coastlinecolor="Black"
    ),
    sliders=[dict(
        active=0,
        steps=steps,
        x=0.1,
        y=0,
        xanchor="left",
        yanchor="top",
        len=0.8
    )],
    updatemenus=[dict(
        buttons=buttons,
        direction="right",
        pad={"r": 10, "t": 10},
        showactive=True,
        type="buttons",
        x=0.1,
        xanchor="left",
        y=1.12,
        yanchor="top"
    )]
)

fig.show()
